In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

headers = {
    'User-Agent': 'Mozilla/5.0'
}

# Daftar kategori dan URL tag Detik yang sesuai
kategori_urls = {
    'Non-Sepak Bola': [
        'https://www.detik.com/tag/bulutangkis/',
        'https://www.detik.com/tag/basket/',
        'https://www.detik.com/tag/voli/'
    ],
    'Liga Inggris': ['https://www.detik.com/tag/liga-inggris/'],
    'Liga Indonesia': ['https://www.detik.com/tag/liga-1/'],
    'Liga Spanyol': ['https://www.detik.com/tag/la-liga/'],
    'Liga Italia': ['https://www.detik.com/tag/serie-a/']
}

data_berita = []
nomor = 1

# Fungsi untuk scraping per kategori
def scrape_kategori(kategori, urls, jumlah_target=20):
    global nomor
    berita_kategori = []
    for url in urls:
        res = requests.get(url, headers=headers)
        soup = BeautifulSoup(res.text, 'html.parser')
        links = soup.select("article a")
        
        for a in links:
            link = a.get('href')
            if not link or not link.startswith('http'):
                continue
            
            try:
                r_artikel = requests.get(link, headers=headers)
                soup_artikel = BeautifulSoup(r_artikel.text, 'html.parser')
                paragraphs = soup_artikel.select('div.detail__body-text p')
                isi_teks = ' '.join(p.get_text(strip=True) for p in paragraphs)
                
                if len(isi_teks.split()) >= 100:
                    berita_kategori.append({
                        'No': nomor,
                        'Text': isi_teks,
                        'URL': link,
                        'Kategori': kategori
                    })
                    nomor += 1
                    if len(berita_kategori) >= jumlah_target:
                        return berita_kategori
                time.sleep(1)
            except Exception:
                continue
    return berita_kategori

# Scrape semua kategori
for kategori, urls in kategori_urls.items():
    hasil = scrape_kategori(kategori, urls, jumlah_target=20)
    data_berita.extend(hasil)

# Simpan ke DataFrame
df_total = pd.DataFrame(data_berita)
df_total.head()


,No,Text,URL,Kategori
0,1,Turnamen bulutangkis usia muda bertajukBupati ...,https://sport.detik.com/raket/d-7906512/semoga...,Non-Sepak Bola
1,2,Pebulutangkis tunggal putra IndonesiaAnthony S...,https://sport.detik.com/raket/d-7906006/alwi-u...,Non-Sepak Bola
2,3,Wakil Ketua Umum I PP PBSITaufik Hidayatbelum ...,https://sport.detik.com/raket/d-7905795/taufik...,Non-Sepak Bola
3,4,Pebulutangkis ganda putra RIFajar Alfianmemato...,https://sport.detik.com/raket/d-7904470/target...,Non-Sepak Bola
4,5,Indonesia Open 2025yang akan diselenggarakan p...,https://sport.detik.com/raket/d-7903675/harga-...,Non-Sepak Bola
